<h2 style="color:#ff6961"> Notes - THIS DEMO IS DEDICATED TO MOTOR</h2> <ol  style="color:#A7C7E7">
   
  <li>All the data is random and ultimatley useless - all created with random samples</li>
  <li>Sample sizes can be set</li>
  <li>Several lines of business</li>
  <li>In fututre could set 1 LOB to have more a more realistic range of sales / purchase price / conversion rate</li>  
  <li>Simply a demo over different markets, seperate packs could (and may) be create for each LOB</li>  
  <li>Stratify Purchase Price & Premium by LOB</li>
  <li>The geographical component is random Lats & Longs, so when reverse geocoding may not have a post code if say it's in the Adriatic sea</li>
</ol> 



<h2 style="color:#ff6961"> Works in Progress </h2> <ol  style="color:#A7C7E7">
   
  <li>Stratify the data better (by LOB in particular) to better relect realistic scenarios</li>
  <li>Example - for commercial property setting the value of the individual site, use external data</li>

  <li>Not worthwhile spending time here Going between rpy2 (using Python in R, seperate notebooks for now) - Easier in Data bricks %R, %sql, %Python magic commands</li>
  <li>Include some outliers for testing (so can create a control table to flag or drop these fields)</li>


</ol> 

<h2 style="color:#D198B7">Read in Engine to support custom made packages </h2>

In [1]:
source("C:/Users/alexz/OneDrive/Documents/Git/Engine/Engine.R")

<h2 style="color:#D198B7">Set Paths </h2>

In [2]:
out_path = "OneDrive/Documents/AGCS/Data/Train_Test_Motor.csv"

<h3 style="color:#98D1B2 ">Load in required libraries</h3>

In [3]:
# List of packages required for the project
packs <- c('dplyr', 'tidyr', 'data.table', 'lubridate', 'ids', 'randgeo', 'tidygeocoder')

In [4]:
install_load_packages(packs)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'data.table'


The following objects are masked from 'package:dplyr':

    between, first, last



Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




$dplyr
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

$tidyr
[1] "tidyr"     "dplyr"     "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"     

$data.table
 [1] "data.table" "tidyr"      "dplyr"      "stats"      "graphics"  
 [6] "grDevices"  "utils"      "datasets"   "methods"    "base"      

$lubridate
 [1] "lubridate"  "data.table" "tidyr"      "dplyr"      "stats"     
 [6] "graphics"   "grDevices"  "utils"      "datasets"   "methods"   
[11] "base"      

$ids
 [1] "ids"        "lubridate"  "data.table" "tidyr"      "dplyr"     
 [6] "stats"      "graphics"   "grDevices"  "utils"      "datasets"  
[11] "methods"    "base"      

$randgeo
 [1] "randgeo"    "ids"        "lubridate"  "data.table" "tidyr"     
 [6] "dplyr"      "stats"      "graphics"   "grDevices"  "utils"     
[11] "datasets"   "methods"    "base"      

$tidygeocoder
 [1] "tidygeocoder" "randgeo"      "ids"          "lubridate"    "data.table"  
 [6] "tidyr"        "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"

In [5]:
# Set Sample Size
sample_size = 100000

<h3 style="color:#98D1B2 ">Create Dummy Data - DataFrame</h3>

In [6]:
# Create Vector of factors with colNames
colNames = c("Customer_ID", "Purchase_Date", "Cover_Start_Date", "LOB", "Sale_Flag", "Purchase_Price",
             "Claims_Count", "Period_of_Cover", "Premium", "Age", "Broker")

# Create a Empty DataFrame with 0 rows and n columns (set in sample_size)
df = data.frame(matrix(nrow = sample_size, ncol = length(colNames))) 

<h3 style="color:#98D1B2 ">Create Dummy Data - populate DataFrame</h3>

In [7]:
# Generate Random Customer IDs
df$Customer_ID <- random_id(sample_size)

In [8]:
# SET date components:
s_year <- 2022
s_month <- 01 
s_day <- 01

dt_start <- paste(s_year, s_month, s_day, sep = '-')
dt_start <- as.Date(ymd(dt_start))

current_dt <- as.Date(today())

td <- current_dt - dt_start #time difference

In [9]:
# Assign column names
colnames(df) = colNames

date_range <- as.Date('2022-01-01') - as.Date('2023-01-01') # date Range
date_vec <- as.Date('2023-01-01') + sample(0:td, sample_size, replace = TRUE) # vector of dates

df$Purchase_Date <- date_vec
df$Cover_Start_Date = date_vec + sample(1:365, replace = TRUE) # start dates up to a year of purchase date - random

Warning message in unclass(e1) + unclass(e2):
"longer object length is not a multiple of shorter object length"


In [10]:
# Lines of Business
lob_list <- c('Motor')
df$LOB <- sample(lob_list, sample_size, replace=TRUE)

In [11]:
# Sale Flag
df$Sale_Flag <- sample(0:1, sample_size, replace=TRUE)

In [12]:
# Purchase Price
df$Purchase_Price <- sample(10:100000, sample_size, replace=TRUE)

In [13]:
# Premium
df$Premium <- sample(10:1000, sample_size, replace=TRUE)

In [14]:
# Claims Count
# If there is a sale - there is a possibility of a claim - otherwise it must default to zero
df$Claims_Count <- ifelse(df$Sale_Flag == 1, 
                      sample(0:1, sample_size, replace = T),
                      0)

In [15]:
# Period of cover by months
sample_input <- c(12, 24, 36, 48)
df$Period_of_Cover <- sample(sample_input, sample_size, TRUE)

In [16]:
# Generate random Lats & Longs
# Can then Reverse Geocode to get Postcode
# Slow but an example - other ways to approach 
# This is for demo
lats_longs <- rg_position(sample_size)
lats_longs <- as.data.frame(t(data.frame(lats_longs)))

In [17]:
names(lats_longs) <- c('Latitude', 'Longitude')

df$Latitude <- lats_longs$Latitude
df$Longitude <- lats_longs$Longitude

In [ ]:
start_time <- Sys.time()
# get post_codes 
# if not in the arctic or lake Victoria
# very slow - free but time cost
reverse <- lats_longs %>%
  reverse_geocode(lat = Latitude, long = Longitude, method = 'osm',
                  address = address_found, full_results = TRUE)

Passing 49,802 coordinates to the Nominatim single coordinate geocoder



In [ ]:
end_time <- Sys.time()
end_time - start_time
end_time

In [ ]:
#df$Age <- sample(18, 80, TRUE)
df$Age <- sample(18:80, sample_size, replace=TRUE)

In [ ]:
# Lines of Business
broker_list <- c('london_ins', 'some_syndicate', 'some_mga')
df$Broker <- sample(broker_list, sample_size, replace=TRUE)

In [ ]:
head(df)

In [ ]:
fwrite(df, output_path)